In [120]:
from tkinter import *
import tkinter as tk
from tkinter import Tk,Frame,Label,Entry,Button,messagebox
from tkinter import ttk
import openpyxl
import pandas as pd
from openpyxl import Workbook,load_workbook
from datetime import date,datetime,timedelta
from PIL import ImageTk, Image

def obter_vagas_excel_filtro(event=None):
    global registros_vagas

    planilha = openpyxl.load_workbook("Dados.xlsx")
    aba = planilha["Reservas"]
    data_para_busca = entry_data_pesquisa.get()

    registros_vagas = {}

    for secao in range(1,5):
        registros_vagas[chr(64+secao)] = {} #converte em caracter ASCII para letra

        for lugar in range(1,6):
            registros_vagas[chr(64+secao)][str(lugar)] = "--    "
        
    for linha in aba.iter_rows(min_row=2,values_only=True):
        data_entrada = str(linha[2])
        data_saida = str(linha[4])
        status = str(linha[9])

        if data_entrada == data_para_busca and status=="Reservado":
            secao = str(linha[7])
            numero_lugar = str(linha[6])
            registros_vagas[secao][numero_lugar] = "X    "

    return registros_vagas

def obter_hora_atual(widget):
    hora_atual = datetime.now().strftime("%H:%M")
    widget["Hora de Entrada"].delete(0,END)
    widget["Hora de Entrada"].insert(0,hora_atual)

def exibir_painel_vagas():    
    global entry_data_pesquisa, janela_painel, registros_vagas,frame_reserva,widgets_entrada
    
    janela_painel = Toplevel()
    janela_painel.title("Painel de Estacionamento")

    frame_busca = Frame(janela_painel,bg='#ffffff')
    frame_busca.pack(fill="both",expand=True)

    entry_data_pesquisa = Entry(frame_busca,font=("Arial",20),justify=CENTER)
    entry_data_pesquisa.grid(row=0,column=0,padx=5,pady=10)

    data_atual = datetime.now().strftime("%d/%m/%Y")
    
    entry_data_pesquisa.insert(0,data_atual)

    def buacar_vagas():
        global registros_vagas
        registros_vagas = obter_vagas_excel_filtro()
        atualizar_vagas_no_painel(registros_vagas)

    button_buscar = Button(frame_busca,text="Buscar",
                            command=buacar_vagas,font=("Arial",16))
    button_buscar.grid(row=0,column=1,padx=5,pady=5)

    frame_reserva = Frame(janela_painel,bg="#ffffff")
    frame_reserva.pack(fill="both",expand=True,pady=20)

    colunas = ["CPF","Nome","Data de Entrada","Hora de Entrada","Número da Vaga","Seção"]
    widgets_entrada = {}
    caminho_arquivo = "Dados.xlsx"
    planilha_nome = "Clientes"
    df = pd.read_excel(caminho_arquivo,sheet_name=planilha_nome)

    combo_cpf_var = StringVar()
    combo_nome_var = StringVar()

    def cpf_selecionado(event):
        cpf_atual = combo_cpf_var.get()
        df_filtrado = df[df['CPF'] == cpf_atual]
        if not df_filtrado.empty:
            nome_correspondente = df_filtrado['Nome'].values[0]
            combo_nome_var.set(nome_correspondente)
        else:
            print(f"CPF {cpf_atual} não encontrado.")

    for i,coluna in enumerate(colunas):
        rotulo = Label(frame_reserva,text=coluna,font=("Arial",14),bg="#ffffff")
        rotulo.grid(row=i,column=0,padx=5,pady=5)

        if coluna == "CPF":
            valores_combo = df["CPF"].tolist()

            entrada = ttk.Combobox(frame_reserva,
                                    textvariable=combo_cpf_var,
                                    values=valores_combo,
                                    font=("Arial",14))
            entrada.bind("<<ComboboxSelected>>",cpf_selecionado)
            entrada.bind("<FocusOut>",cpf_selecionado)

        elif coluna == "Nome":
            valores_combo = df["Nome"].tolist()

            entrada = ttk.Combobox(frame_reserva,
                                    textvariable=combo_nome_var,
                                    values=valores_combo,
                                    font=("Arial",14))

        elif coluna == "Número da Vaga":
            valores_combo = [str(num) for num in range(1,6)]
            entrada = ttk.Combobox(frame_reserva,
                                    values=valores_combo,
                                    font=("Arial",14),
                                    state="readonly")
        elif coluna == "Seção":
            valores_combo = ["A","B","C","D"]
            entrada = ttk.Combobox(frame_reserva,
                                    values=valores_combo,
                                    font=("Arial",14),
                                    state="readonly")
        else:
            entrada = Entry(frame_reserva,font=("Arial",14))

        entrada.grid(row=i,column=1,padx=5,pady=5)
        widgets_entrada[coluna] = entrada

    frame_reserva.grid_columnconfigure(0,weight=1)
    frame_reserva.grid_columnconfigure(1,weight=1)

    botao_salvar_reserva = Button(frame_reserva,
                                    text="Salvar Reserva",
                                    command=salvar_reserva,
                                    width=30)
    botao_salvar_reserva.grid(row=i+1,column=0,padx=5,pady=5,sticky="EW")

    botao_salvar_automaticamente = Button(frame_reserva,
                                    text="Escolher Automaticamente",
                                    command=escolhe_vaga_automaticamente,
                                    width=20)
    botao_salvar_automaticamente.grid(row=i+1,column=1,padx=5,pady=5,sticky="EW")

    widgets_entrada["Data de Entrada"].delete(0,END)
    data_atual = datetime.now().strftime("%d/%m/%Y")
    widgets_entrada["Data de Entrada"].insert(0,data_atual)

    obter_hora_atual(widgets_entrada)

    def atualizar_vagas_no_painel(registros_vagas):
        for widget in janela_painel.winfo_children():
            if widget not in (frame_busca,frame_reserva):
                widget.destroy()
        
        label_cabecalho = Label(janela_painel,
                                text="Seção/Vaga 1     2     3     4     5",
                                font=("Arial",16))
        label_cabecalho.pack()

        for secao,v in registros_vagas.items():
            frame_secao  = Frame(janela_painel)
            label_secao = Label(frame_secao,
                                text="{:<19}".format(secao),
                                font=("Arial",16))
            label_secao.pack(side=LEFT)

            for lugar in range(1,6):
                status = v[str(lugar)]
    
                label_lugar = Label(frame_secao,text=status,
                                    font=("Arial",16))
                label_lugar.pack(side=LEFT)
            
            frame_secao.pack()
    
    registros_vagas = obter_vagas_excel_filtro()
    label_cabecalho = Label(janela_painel,
                            text="Seção/Vaga 1     2     3     4     5",
                            font=("Arial",16))
    label_cabecalho.pack()

    if registros_vagas is not None:
        for secao,v in registros_vagas.items():
            frame_secao  = Frame(janela_painel)
            label_secao = Label(frame_secao,
                                text="{:<19}".format(secao),
                                font=("Arial",16))
            label_secao.pack(side=LEFT)

            for lugar in range(1,6):
                status = v[str(lugar)]

                label_lugar = Label(frame_secao,text=status,
                                    font=("Arial",16))
                label_lugar.pack(side=LEFT)
                
            frame_secao.pack()

    #buscar_vagas()

    janela_painel.update_idletasks()
    largura = janela_painel.winfo_width()
    altura = janela_painel.winfo_height()
    pos_x = (janela_painel.winfo_screenwidth() // 2) - (largura // 2)
    pos_y = (janela_painel.winfo_screenheight() // 2) - (altura // 2)
    janela_painel.geometry("{}x{}+{}+{}".format(largura,altura,pos_x,pos_y))

def salvar_reserva():
    global widgets_entrada,frame_cadastro,vagas

    cpf_cliente = widgets_entrada['CPF'].get()
    nome_cliente = widgets_entrada['Nome'].get()
    data_entrada_cliente = widgets_entrada['Data de Entrada'].get()
    hora_cliente = widgets_entrada['Hora de Entrada'].get()
    numero_vaga = widgets_entrada['Número da Vaga'].get()
    bloco = widgets_entrada['Seção'].get()
    status = "Reservado"

    workbook = openpyxl.load_workbook("Dados.xlsx")
    planilha = workbook["Reservas"]

    vaga_encontrada = False
    
    data_entrada_datetime = datetime.strptime(data_entrada_cliente,"%d/%m/%Y")
    
    for linha in planilha.iter_rows(min_row=2,values_only=True):
        data_entrada_excel = datetime.strptime(str(linha[2]), "%d/%m/%Y")
        data_saida_excel = None

        if linha[4] and str(linha[4]).strip():
            try:
                data_saida_excel = datetime.strptime(str(linha[4]),"%d/%m/%Y")
            except ValueError:
                pass
            
        vaga_reservada = str(linha[6])
        bloco_reservada = str(linha[7])
        
        if data_entrada_excel == data_entrada_datetime and (not data_saida_excel) and vaga_reservada == str(numero_vaga) and bloco_reservada == bloco:
            messagebox.showwarning("Reserva já existe","Esta vaga não esta disponivel.")
            vaga_encontrada = True
            janela_painel.destroy()
            exibir_painel_vagas()
            return
        
        if vaga_encontrada:
            return
        
        linha_vazia = 2

        while planilha.cell(row=linha_vazia,column=1).value is not None:
            linha_vazia += 1

    planilha.cell(row=linha_vazia,column=1).value = cpf_cliente
    planilha.cell(row=linha_vazia,column=2).value = nome_cliente
    planilha.cell(row=linha_vazia,column=3).value = data_entrada_cliente
    planilha.cell(row=linha_vazia,column=4).value = hora_cliente
    planilha.cell(row=linha_vazia,column=5).value = "-"
    planilha.cell(row=linha_vazia,column=6).value = "-"
    planilha.cell(row=linha_vazia,column=7).value = numero_vaga
    planilha.cell(row=linha_vazia,column=8).value = bloco
    planilha.cell(row=linha_vazia,column=9).value = 0
    planilha.cell(row=linha_vazia,column=10).value = status

    workbook.save("Dados.xlsx")
    
    messagebox.showinfo("Reserva Efatuada", "Reserva efetuada com sucesso")
    janela_painel.destroy()
    exibir_painel_vagas()
            
def escolhe_vaga_automaticamente():
    global widgets_entrada,frame_cadastro,vagas

    cpf_cliente = widgets_entrada['CPF'].get()
    nome_cliente = widgets_entrada['Nome'].get()
    data_entrada_cliente = widgets_entrada['Data de Entrada'].get()
    hora_cliente = widgets_entrada['Hora de Entrada'].get()
    numero_vaga = widgets_entrada['Número da Vaga'].get()
    bloco = widgets_entrada['Seção'].get()
    status = "Reservado"

    workbook = openpyxl.load_workbook("Dados.xlsx")
    planilha = workbook["Reservas"]
    
    data_entrada_datetime = datetime.strptime(data_entrada_cliente,"%d/%m/%Y")

    vaga_disponivel = None
    bloco_disponivel = None

    if not numero_vaga and not bloco:

        vagas_ocupadas = []
    
        for linha in planilha.iter_rows(min_row=2,values_only=True):
            
            data_entrada_excel = None
            data_saida_excel = None

            if linha[2] and str(linha[2]).strip():
                try:
                    data_entrada_excel = datetime.strptime(str(linha[2]),"%d/%m/%Y")
                except ValueError:
                    pass
                        
            if linha[4] and str(linha[4]).strip():
                try:
                    data_saida_excel = datetime.strptime(str(linha[4]),"%d/%m/%Y")
                except ValueError:
                    pass
            
            if (data_saida_excel is None and data_entrada_excel.date()== data_entrada_datetime.date()) or (data_saida_excel and data_entrada_excel.date() == data_entrada_datetime.date()):
                vagas_ocupadas.append((linha[6],linha[7]))

        for bloco_canditado in ['A','B','C','D']:
            for vaga_canditada in range(1,6):
                vaga_canditada = str(vaga_canditada)

                if(vaga_canditada,bloco_canditado) not in vagas_ocupadas:
                    vaga_disponivel = vaga_canditada
                    bloco_disponivel = bloco_canditado
                    break

            if vaga_disponivel:
                break

        if not vaga_disponivel:
            messagebox.showwarning("Todas as Vagas Ocupadas","Não há vagas disponíveis")
            return
        
        numero_vaga = vaga_disponivel
        bloco = bloco_disponivel
        
    else:
        for linha in planilha.iter_rows(min_row=2,values_only=True):
            
            if linha[2] and str(linha[2]).strip():
                try:
                    data_entrada_excel = datetime.strptime(str(linha[2]),"%d/%m/%Y")
                except ValueError:
                    pass

            if linha[4] and str(linha[4]).strip():
                try:
                    data_saida_excel = datetime.strptime(str(linha[4]),"%d/%m/%Y")
                except ValueError:
                    pass
            if linha[6] == numero_vaga and linha[7] == bloco and ((data_saida_excel is None and data_entrada_excel.date() == data_entrada_datetime.date()) or (data_saida_excel and data_saida_excel.date() == data_entrada_datetime.date())):
                messagebox.showwarning("Vaga Ocupada","A vaga selecionada não esta disponivel")
                return
    
    linha_vazia = 2

    while planilha.cell(row=linha_vazia,column=1).value is not None:
        linha_vazia += 1
        
    planilha.cell(row=linha_vazia,column=1).value = cpf_cliente
    planilha.cell(row=linha_vazia,column=2).value = nome_cliente
    planilha.cell(row=linha_vazia,column=3).value = data_entrada_cliente
    planilha.cell(row=linha_vazia,column=4).value = hora_cliente
    planilha.cell(row=linha_vazia,column=5).value = "-"
    planilha.cell(row=linha_vazia,column=6).value = "-"
    planilha.cell(row=linha_vazia,column=7).value = numero_vaga
    planilha.cell(row=linha_vazia,column=8).value = bloco
    planilha.cell(row=linha_vazia,column=9).value = 0
    planilha.cell(row=linha_vazia,column=10).value = status

    workbook.save("Dados.xlsx")

    for widget in widgets_entrada.values():
        widget.delete(0,tk.END)

    messagebox.showinfo("Reserva Efatuada", "Reserva efetuada com sucesso")

    janela_painel.destroy()
    exibir_painel_vagas()    

def tela_clientes():
    janela_cadastro_cliente = tk.Toplevel()
    janela_cadastro_cliente.title("Clientes")
    janela_cadastro_cliente.configure(bg="#ffffff")

    clientes = {}

    filtro_frama = tk.Frame(janela_cadastro_cliente,bg="#fff")
    filtro_frama.pack(pady=10)

    filtro_label = tk.Label(filtro_frama,
                            text="Filtrar por nome: ",
                            bg="#ffffff",
                            font="Arial 14")
    filtro_label.grid(row=0,column=0,padx=5,pady=3)
    filtro_entry = tk.Entry(filtro_frama,width=60)
    filtro_entry.grid(row=0,column=1,padx=5,pady=3)

    frame_treeview = tk.Frame(janela_cadastro_cliente,bg="#ffffff")
    frame_treeview.pack(fill=tk.BOTH,expand=True)

    style = ttk.Style()
    style.configure("Treeview",font=("Arial",14),rowheight=28)

    clientes_treeview = ttk.Treeview(frame_treeview)

    clientes_treeview['columns'] = ('CPF','Nome')

    clientes_treeview.heading('#0',text='',anchor='w')
    clientes_treeview.column('#0',stretch='NO',width=0)
    clientes_treeview.heading('CPF',text='CPF',anchor='w')
    clientes_treeview.column('CPF',anchor='w')
    clientes_treeview.heading('Nome',text='Nome',anchor='w')
    clientes_treeview.column('Nome',anchor='w')

    clientes_treeview.pack(side=tk.LEFT,fill=tk.BOTH,expand=True)

    scrollbar = ttk.Scrollbar(frame_treeview,orient='vertical',command=clientes_treeview.yview)
    scrollbar.pack(side=tk.RIGHT,fill=tk.Y)

    clientes_treeview.configure(yscrollcommand=scrollbar.set)

    cadastro_frame = Frame(janela_cadastro_cliente,bg="#ffffff")
    cadastro_frame.pack(pady=10)

    cpf_label = Label(cadastro_frame,text='CPF: ',font=("Arial",14),bg="#ffffff")
    cpf_label.grid(row=0,column=0,padx=5,pady=3)    
    cpf_entry = Entry(cadastro_frame,font=("Arial",14),width=30)
    cpf_entry.grid(row=0,column=1,padx=5,pady=3)

    nome_label = Label(cadastro_frame,text='Nome: ',font=("Arial",14),bg="#ffffff")
    nome_label.grid(row=1,column=0,padx=5,pady=3)    
    nome_entry = Entry(cadastro_frame,font=("Arial",14),width=30)
    nome_entry.grid(row=1,column=1,padx=5,pady=3)

    def filtrar_clientes():
        filtro = filtro_entry.get().lower()
        clientes_treeview.delete(*clientes_treeview.get_children())

        for produto in clientes.values():
            nome = produto['Nome'].lower()

            if filtro in nome:
                cpf = produto['CPF']
                nome = produto['Nome']
                clientes_treeview.insert('', 'end', values=(cpf,nome))
        
    filtro_entry.bind("<KeyRelease>",lambda event: filtrar_clientes())

    def carregar_clientes_do_excel():
        clientes_treeview.delete(*clientes_treeview.get_children())
        caminho_arquivo = 'Dados.xlsx'
        planilha_nome = 'Clientes'
        
        try:
            workbook = load_workbook(filename=caminho_arquivo)
            sheet = workbook[planilha_nome]

            for row in sheet.iter_rows(min_row=2,values_only=True):
                cpf, nome = row
                clientes[cpf] = {'CPF':cpf,'Nome':nome}
                clientes_treeview.insert('', 'end', values=(cpf,nome))

        except FileExistsError:
            messagebox.showwarning("Carregar Cliente", 
                                "Arquivo Dados.xlsx não encontrado",
                                parent=janela_cadastro_cliente)


    carregar_clientes_do_excel()

    def preencher_campos(event):
        selected_item = clientes_treeview.focus()

        if selected_item:
            cpf = clientes_treeview.item(selected_item, 'values')[0]
            nome = clientes_treeview.item(selected_item, 'values')[1]

            cpf_entry.delete(0,tk.END)
            nome_entry.delete(0,tk.END)
            
            cpf_entry.insert(tk.END,cpf)
            nome_entry.insert(tk.END,nome)
    
    clientes_treeview.bind("<Double-1>",preencher_campos)

    def cadastrar_cliente():
        cpf = cpf_entry.get()
        nome = nome_entry.get()

        if cpf and nome:
            if cpf not in clientes:
                clientes[cpf] = {'CPF':cpf,'Nome':nome}
                clientes_treeview.insert('', 'end', values=(cpf,nome))
                messagebox.showinfo("Cadastro de Cliente","Cliente cadastrado com sucesso",parent=janela_cadastro_cliente)

                limpa_campos()
                salvar_cliente_excel()
            
            else:
                messagebox.showwarning("Cadastro de Cliente","Cliente já cadastrado",parent=janela_cadastro_cliente)

        else:
            messagebox.showwarning("Cadastro de Cliente","Preencha todos os campos",parent=janela_cadastro_cliente)

    cadastrar_button = Button(janela_cadastro_cliente,
                                text="Cadastrar",
                                command=cadastrar_cliente,
                                font=("Arial",14))
    cadastrar_button.pack(side=tk.LEFT,padx=5,pady=3)
    
    def alterar_cliente():
        cpf = cpf_entry.get()
        nome = nome_entry.get()

        if cpf and nome:

            caminho_arquivo = 'Dados.xlsx'
            planhlia_nome = 'Clientes'

            try:
                workbook = load_workbook(filename=caminho_arquivo)
                sheet = workbook[planhlia_nome]

                for row in sheet.iter_rows(min_row=2):

                    if row[0].value == cpf:
                        row[1].value = nome
                        break
            
                workbook.save(filename=caminho_arquivo)

                messagebox.showinfo("Alterar Cliente","Cliente alterado com sucesso",parent=janela_cadastro_cliente)
                limpa_campos()
                carregar_clientes_do_excel()

            except FileExistsError:
                messagebox.showwarning("Alterar Cliente", "Dados.xlsx não encontrado",parent=janela_cadastro_cliente)
        
        else:
            messagebox.showwarning("Alterar Cliente", "Preencha todos os campos",parent=janela_cadastro_cliente)

    alterar_button = Button(janela_cadastro_cliente,
                                text="Alterar",
                                command=alterar_cliente,
                                font=("Arial",14))
    alterar_button.pack(side=tk.LEFT,padx=5,pady=3)
    
    def excluir_cliente():
        cpf = cpf_entry.get()

        if cpf:
            caminho_arquivo = 'Dados.xlsx'
            planilha_nome = 'Clientes'
            
            try:
                workbook = load_workbook(filename=caminho_arquivo)
                sheet = workbook[planilha_nome]

                for row in sheet.iter_rows(min_row=2):

                    if row[0].value == cpf:
                        sheet.delete_rows(row[0].row)
                        break
            
                workbook.save(filename=caminho_arquivo)

                messagebox.showinfo("Excluir Cliente","Cliente alterado com sucesso",parent=janela_cadastro_cliente)
                limpa_campos()
                carregar_clientes_do_excel()

            except FileExistsError:
                messagebox.showwarning("Excluir Cliente", "Dados.xlsx não encontrado",parent=janela_cadastro_cliente)
        
        else:
            messagebox.showwarning("Excluir Cliente", "Preencha o campo CPF",parent=janela_cadastro_cliente)

    excluir_button = Button(janela_cadastro_cliente,
                                text="Excluir",
                                command=excluir_cliente,
                                font=("Arial",14))
    excluir_button.pack(side=tk.LEFT,padx=5,pady=3)
    
    janela_cadastro_cliente.update_idletasks()
    largura = janela_cadastro_cliente.winfo_width()
    altura = janela_cadastro_cliente.winfo_height()
    pos_x = (janela_cadastro_cliente.winfo_screenwidth() // 2) - (largura // 2)
    pos_y = (janela_cadastro_cliente.winfo_screenheight() // 2) - (altura // 2)
    janela_cadastro_cliente.geometry("{}x{}+{}+{}".format(largura,altura,pos_x,pos_y))


    def limpa_campos():
        cpf_entry.delete(0,tk.END)
        nome_entry.delete(0,tk.END)

    def salvar_cliente_excel():
        caminho_arquivo = 'Dados.xlsx'
        planilha_nome = 'Clientes'
        
        try:
            workbook = load_workbook(filename=caminho_arquivo)
        except FileExistsError:
            workbook = Workbook()
        
        sheet = workbook[planilha_nome]

        for cliente in clientes.values():
            cpf = cliente['CPF']
            nome = cliente['Nome']
            cliente_existe = False

            for row in sheet.iter_rows(min_row=2,values_only=True):
                if row[0] == cpf:
                    cliente_existe = True
                    break
            
            if not cliente_existe:
                max_row = sheet.max_row + 1
                sheet.cell(row=max_row, column=1, value=cpf)
                sheet.cell(row=max_row, column=2, value=nome)

        workbook.save(filename=caminho_arquivo)

entry_data_pesquisa = None
windows_painel = None
registros_vagas = {}

def ajustar_largura_colunas(treeview_reservas,colunas,planilha_reservas):
    larguras = {col:len(col) for col in colunas}

    for linha in planilha_reservas.iter_rows(min_row=2,values_only=True):
        for col,valor in zip(colunas,linha):
            if valor is not None:
                larguras[col] = max(larguras[col], len(str(valor)))

        for col in colunas:
            treeview_reservas.column(col, width=larguras[col]*10)

def filtrar(*args):
    treeview_reservas.delete(*treeview_reservas.get_children())

    for linha in planilha_reservas.iter_rows(min_row=2,values_only=True):
        coincide = True

        for col, var in valor_filtro_var.items():
            if var.get() and linha[colunas.index(col)] != var.get():
                coincide = False
                break
        
        if coincide:
            treeview_reservas.insert('', 'end', values=linha)

def reservas():
    global planilha_reservas, treeview_reservas,valor_filtro_var,colunas

    janela_reservas = Toplevel(janela)
    janela_reservas.title('Reservas')
    janela_reservas.configure(bg='white')

    estilo_treeview = ttk.Style()
    estilo_treeview.configure("Treeview",font=("Arial",10))

    colunas = ("CPF do Cliente",
                "Nome do Cliente",
                "Data de Entrada",
                "Hora de Entrada",
                "Data de Saída",
                "Hora de Saída",
                "Número da Vaga",
                "Bloco",
                "Total a Pagar",
                "Status")
    
    caminho_arquivo = "Dados.xlsx"
    planilha_nome = "Reservas"

    workbook = load_workbook(filename=caminho_arquivo)
    planilha_reservas = workbook[planilha_nome]

    valores_unicos = {col:set() for col in colunas}

    for linha in planilha_reservas.iter_rows(min_row=2, values_only=True):
        for col, val in zip(colunas,linha):
            valores_unicos[col].add(val)

    valor_filtro_var = {col: StringVar() for col in colunas}

    for i, col in enumerate(colunas):
        style = ttk.Style()
        style.configure("BW.TLabel",background='white')

        ttk.Label(janela_reservas,
                    text=f"Filtro {col}",
                    style="BW.TLabel").grid(row= i // 3, 
                                            column= 2 * (i % 3),
                                            padx=10, pady=10)
        
        combo = ttk.Combobox(janela_reservas,textvariable=valor_filtro_var[col])
        combo.grid(row= i // 3,column= 2 * (i % 3) + 1,padx=10, pady=10)
        combo['values'] = list(valores_unicos[col])
        
        valor_filtro_var[col].trace('w',filtrar)

    treeview_reservas = ttk.Treeview(janela_reservas,columns=colunas,show='headings')

    for col in colunas:
        treeview_reservas.heading(col, text=col, anchor='center')
    
    treeview_reservas.grid(row=len(colunas) // 3+1, column=0,columnspan=6,sticky='nsew')
    
    for linha in planilha_reservas.iter_rows(min_row=2,values_only=True):
        treeview_reservas.insert("","end",values=linha)

    fechar_button = Button(janela_reservas,
                            text="Fechar",
                            font=("Arial",12),
                            command=janela_reservas.destroy)
    fechar_button.grid(row=len(colunas) // 3 + 2,column=0,columnspan=6,pady=10)

    ajustar_largura_colunas(treeview_reservas,colunas,planilha_reservas)

    def abrir_detalhes(event):
        item_id = treeview_reservas.selection()[0]
        valores = treeview_reservas.item(item_id)["values"]
        
        janela_detalhes = Toplevel(janela_reservas)
        janela_detalhes.title("Detalhes da Reserva")
        janela_detalhes.configure(bg="white")

        entries = []
        
        for i,(col,val) in enumerate(zip(colunas,valores)):
            Label(janela_detalhes,text=col,bg="white").grid(row=i,column=0,padx=10,pady=10)

            if col == 'Status':
                status_var = StringVar()
                entry = ttk.Combobox(janela_detalhes, textvariable=status_var, font="Arial 14")
                entry['values'] = ('Livre','Reservado')
                entry.set(val)

            if col == 'Data de Saída':
                entry = ttk.Entry(janela_detalhes,font="Arial 14")

            if col == 'Hora de Saída':
                entry = ttk.Entry(janela_detalhes,font="Arial 14")            
            
            if col == 'Total a Pagar':
                entry = ttk.Entry(janela_detalhes,font="Arial 14")            
            
            if col == 'Status':
                valores_combo = ["-", "Reservado", "Livre"]
                entry = ttk.Combobox(janela_detalhes,values=valores_combo,state="readonly",font="Arial 14")

            else:
                entry = ttk.Entry(janela_detalhes,font="Arial 14")
                entry.insert(0,val)

            entry.grid(row=i,column=1,padx=10,pady=10)
            entries.append(entry)

        def salvar_alteracoes(item_id, entries):
            workbook = load_workbook(filename=caminho_arquivo)
            worksheet = workbook['Reservas']
            indices_campo_busca = [0,2,6,7]

            for row in worksheet.iter_rows(min_row=2):
                if all(row[i].value == entries[i].get() for i in indices_campo_busca):
                    for cell, entry in zip(row,entries):
                        cell.value = entry.get()
                    break

            workbook.save(caminho_arquivo)
            atualizar_treeview()
        
        Button(janela_detalhes,text="Salvar Alterações",
                command=lambda: salvar_alteracoes(item_id,entries)).grid(row=len(colunas),
                                                                        column=0,padx=10,pady=10)
        

        def atualizar_treeview():
            treeview_reservas.delete(*treeview_reservas.get_children())

            df_reservas = pd.read_excel(caminho_arquivo,sheet_name=planilha_nome)

            for i, row in df_reservas.iterrows():
                treeview_reservas.insert('', 'end', values=list(row))

        def excluir_reserva(item_id,entries):
            workbook = load_workbook(filename=caminho_arquivo)
            worksheet = workbook['Reservas']
            indices_campo_busca = [0,2,6,7]

            for row in worksheet.iter_rows(min_row=2):
                if all(row[i].value == entries[i].get() for i in indices_campo_busca):
                    worksheet.delete_rows(row[0].row)
                    break

            workbook.save(caminho_arquivo)
            atualizar_treeview()        

        Button(janela_detalhes,text="Excluir",
                command=lambda: excluir_reserva(item_id,entries)).grid(row=len(colunas),
                                                                        column=1,padx=10,pady=10)
        def calcular_preco(data_entrada_str,hora_entrada_str,data_saida_str,hora_saida_str):
            
            data_entrada = datetime.strptime(data_entrada_str,'%d/%m/%Y')
            hora_entrada = datetime.strptime(hora_entrada_str,'%H:%M').time()
            data_saida = datetime.strptime(data_saida_str,'%d/%m/%Y')
            hora_saida = datetime.strptime(hora_saida_str,'%H:%M').time()

            entrada = datetime.combine(data_entrada,hora_entrada)
            saida = datetime.combine(data_saida,hora_saida)


            diferenca = saida - entrada
            horas = diferenca.total_seconds() / 3600

            preco = 8 + max(0, horas - 1)

            return preco
        
        def atualizar_preco():

            entry_data_entrada = entries[colunas.index('Data de Entrada')]
            entry_hora_entrada = entries[colunas.index('Hora de Entrada')]
            entry_data_saida = entries[colunas.index('Data de Saída')]
            entry_hora_saida = entries[colunas.index('Hora de Saída')]
            entry_total = entries[colunas.index('Total a Pagar')]

            data_saida_atual = datetime.now().strftime("%d/%m/%Y")
            hora_saida_atual = datetime.now().strftime("%H:%M")

            data_entrada_str = entry_data_entrada.get()
            hora_entrada_str = entry_hora_entrada.get()
            data_saida_str = entry_data_saida.get()
            hora_saida_str = entry_hora_saida.get()
            
            preco = calcular_preco(data_entrada_str,hora_entrada_str,data_saida_str,hora_saida_str)
            
            entry_total.delete(0,tk.END)
            entry_total.insert(0,'{:,.2f}'.format(preco))  

        Button(janela_detalhes,text="Calcular Preço",
                command=atualizar_preco).grid(row=len(colunas)+1,column=0,padx=10,pady=10)
        
        janela_detalhes.update_idletasks()
        largura = janela_detalhes.winfo_width()
        altura = janela_detalhes.winfo_height()
        pos_x = (janela_detalhes.winfo_screenwidth() // 2) - (largura // 2)
        pos_y = (janela_detalhes.winfo_screenheight() // 2) - (altura // 2)
        janela_detalhes.geometry("{}x{}+{}+{}".format(largura,altura,pos_x,pos_y))

    treeview_reservas.bind("<Double-1>",abrir_detalhes)

    janela_reservas.update_idletasks()
    largura = janela_reservas.winfo_width()
    altura = janela_reservas.winfo_height()
    pos_x = (janela_reservas.winfo_screenwidth() // 2) - (largura // 2)
    pos_y = (janela_reservas.winfo_screenheight() // 2) - (altura // 2)
    janela_reservas.geometry("{}x{}+{}+{}".format(largura,altura,pos_x,pos_y))


janela = tk.Tk()
janela.title("Sistema de Controle de Vagas")
janela.configure(bg='white')
janela.attributes("-fullscreen",True)

try:
    imagem_carregada = Image.open("estacionamento.jpg")
    imagem = ImageTk.PhotoImage(imagem_carregada)

    rotulo_imagem = Label(janela)
    rotulo_imagem.image = imagem
    rotulo_imagem.configure(image=imagem)
    rotulo_imagem.pack()

except Exception as e:
    print(f"Erro ao carregar a imagem: {e}")

messagem_label = tk.Label(janela,
                        text="Bem-vindo ao Controle de Vagas Estacionamento",
                        font=("Arial",16),
                        bg='white')
messagem_label.pack(pady=50)

button_frame = tk.Frame(janela,bg='white')
button_frame.pack()

button_painel = tk.Button(button_frame,
                        text="Exibir Painel de Vagas",
                        command=exibir_painel_vagas,
                        font=("Arial",12),
                        width=40)
button_painel.pack(side=tk.TOP,padx=50,pady=10,ipadx=20,ipady=10)

button_clientes = tk.Button(button_frame,
                        text="Clientes",
                        command=tela_clientes,
                        font=("Arial",12),
                        width=40)
button_clientes.pack(side=tk.TOP,padx=50,pady=10,ipadx=20,ipady=10)

button_reservas = tk.Button(button_frame,
                        text="Reservas",
                        command=reservas,
                        font=("Arial",12),
                        width=40)
button_reservas.pack(side=tk.TOP,padx=50,pady=10,ipadx=20,ipady=10)

sair_button = tk.Button(button_frame,
                        text="Sair",
                        font=("Arial",12),
                        command=janela.destroy,
                        width=40)
sair_button.pack(side=tk.TOP,padx=50,pady=10,ipadx=20,ipady=10)

janela.mainloop()

2023-11-19 00:00:00
13:04:00
2023-11-19 00:00:00
17:04:00
2023-11-19 00:00:00
13:04:00
2023-11-19 00:00:00
17:04:00
2023-11-19 00:00:00
13:04:00
2023-11-19 00:00:00
20:04:00
2023-11-19 00:00:00
13:04:00
2023-11-19 00:00:00
20:04:00
